In [51]:
import gzip
import pickle
import torch

from tqdm import tqdm

In [52]:
class Model:
    LEARNING_RATE = 0.01

    def __init__(self, input_dimensions, output_dimensions):
        self.input_dimensions = input_dimensions
        self.output_dimensions = output_dimensions
        self.biases = None  # shape: (1, 10)
        self.weights = None  # shape: (784, 10)
        self.training_set = None
        self.validation_set = None
        self.testing_set = None

    @staticmethod
    def sigmoid(z):
        return 1.0 / (1.0 + torch.exp(-z))

    def load_input(self):
        def _map_data(given_set):
            data = given_set[0]
            tags = given_set[1]
            output = []

            for index in range(len(tags)):
                output += [(torch.from_numpy(data[index]).view(1, self.input_dimensions), tags[index])]

            return output

        with gzip.open("mnist.pkl.gz", "rb") as fd:
            training_set, validation_set, testing_set = pickle.load(fd, encoding='latin')

        self.training_set = _map_data(training_set)
        self.validation_set = _map_data(validation_set)
        self.testing_set = _map_data(testing_set)

    def load_params(self):
        self.biases = torch.rand(1, self.output_dimensions, dtype=torch.float32)  # 1, 784
        self.weights = torch.rand(self.input_dimensions, self.output_dimensions, dtype=torch.float32)  # 784, 10

    def train_online(self, data_set, max_iterations, learning_rate):
        iterations = max_iterations
        all_classified = False

        while not all_classified and iterations > 0:
            iterations -= 1
            all_classified = True
            for input_values, correct_tag in tqdm(data_set, unit=" entries",
                                                  desc=f"Epoch {max_iterations - iterations}/{max_iterations}"):
                expected_result = torch.tensor([1 if i == correct_tag else 0 for i in range(self.output_dimensions)],
                                               dtype=torch.float32)
                output = torch.matmul(input_values, self.weights) + self.biases
                activated_output = Model.sigmoid(output)

                self.weights = self.weights + torch.matmul(input_values.view(self.input_dimensions, 1),
                                                           (expected_result - activated_output).view(1,
                                                                                                     self.output_dimensions)) * learning_rate
                self.biases = self.biases + (expected_result - activated_output) * learning_rate

                if not torch.equal(activated_output, expected_result):
                    all_classified = False

    def train_mini_batch(self, data_set, max_iterations, nr_batches, learning_rate):
        iterations = max_iterations
        all_classified = False
        while not all_classified and iterations > 0:
            iterations -= 1
            all_classified = True
            batch_size = len(data_set) // nr_batches

            for batch_index in tqdm(range(nr_batches), unit=" mini batches",
                                    desc=f"Epoch {max_iterations - iterations}/{max_iterations}"):
                delta_weights = torch.zeros(self.input_dimensions, self.output_dimensions, dtype=torch.float32)
                delta_biases = torch.zeros(self.output_dimensions, dtype=torch.float32)
                batch = data_set[batch_index * batch_size: (batch_index + 1) * batch_size]

                for input_values, correct_tag in batch:
                    expected_result = torch.tensor(
                        [1 if i == correct_tag else 0 for i in range(self.output_dimensions)], dtype=torch.float32)
                    output = torch.matmul(input_values, self.weights) + self.biases
                    activated_output = Model.sigmoid(output)

                    delta_weights = delta_weights + torch.matmul(input_values.view(self.input_dimensions, 1),
                                                                 (expected_result - activated_output).view(1,
                                                                                                           self.output_dimensions)) * learning_rate
                    delta_biases = delta_biases + (expected_result - activated_output) * learning_rate

                    if not torch.equal(activated_output, expected_result):
                        all_classified = False

                self.weights += delta_weights
                self.biases += delta_biases

    def predict(self, input_values):
        output = torch.matmul(input_values, self.weights) + self.biases
        activated_output = Model.sigmoid(output)
        if activated_output.sum() > 1:
            return torch.argmax(output).item()
        return torch.argmax(activated_output).item()

    def test_model(self, data_set):
        wrong_predictions = 0
        correct_predictions = 0

        for input_values, correct_tag in data_set:
            predicted_value = self.predict(input_values)
            if predicted_value == correct_tag:
                correct_predictions += 1
            else:
                wrong_predictions += 1

        print(f"Correct: {correct_predictions}, "
              f"Wrong: {wrong_predictions},"
              f" Total: {correct_predictions + wrong_predictions}, "
              f"Accuracy: {int(correct_predictions / (correct_predictions + wrong_predictions) * 10000.) / 100}%\n")

In [53]:
model = Model(784, 10)
model.load_input()
model.load_params()
print('Results BEFORE training:')
model.test_model(model.testing_set)

model.train_online(model.training_set, 10, 0.05)

print('Results on testing set AFTER online training:')
model.test_model(model.testing_set)

print('Results on validation set AFTER online training:')
model.test_model(model.validation_set)

Results BEFORE training:
Correct: 1345, Wrong: 8655, Total: 10000, Accuracy: 13.45%


Epoch 10/10: 100%|██████████| 50000/50000 [00:01<00:00, 34351.79 entries/s]


Results on testing set AFTER online training:
Correct: 8943, Wrong: 1057, Total: 10000, Accuracy: 89.43%

Results on validation set AFTER online training:
Correct: 8982, Wrong: 1018, Total: 10000, Accuracy: 89.82%


In [54]:
model = Model(784, 10)
model.load_input()
model.load_params()

print('Results BEFORE training:')
model.test_model(model.testing_set)

model.train_mini_batch(model.training_set, 10, 256, Model.LEARNING_RATE)

print('Results on testing set AFTER mini-batch training:')
model.test_model(model.testing_set)

print('Results on validation set AFTER mini-batch training:')
model.test_model(model.validation_set)

Results BEFORE training:
Correct: 592, Wrong: 9408, Total: 10000, Accuracy: 5.92%


Epoch 10/10: 100%|██████████| 256/256 [00:01<00:00, 176.51 mini batches/s]


Results on testing set AFTER mini-batch training:
Correct: 9050, Wrong: 950, Total: 10000, Accuracy: 90.5%

Results on validation set AFTER mini-batch training:
Correct: 9135, Wrong: 865, Total: 10000, Accuracy: 91.35%
